In [1]:
from key_extraction import keywordExtractor
from transformers import ElectraModel, ElectraTokenizerFast
import numpy as np
import pandas as pd

# load model and tokenizer
name = "monologg/koelectra-base-v3-discriminator"
model = ElectraModel.from_pretrained(name)
tokenizer = ElectraTokenizerFast.from_pretrained(name)

# load keywordExtractor
key = keywordExtractor(model,tokenizer,dir='data/preprocess/eng_han.csv')

# load food data
scraping_result = pd.read_csv('data/food_data.csv',encoding='cp949')
print('음식 데이터 수 : ', len(scraping_result))
print('')
scraping_result.head()

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


음식 데이터 수 :  46



,food_name,food_category,food_description
0,야채김밥,김밥,김에 밥과 다양한 야채를 넣어서 만든 한국의 전통적인 간단한 김밥
1,치즈김밥,김밥,김에 밥과 치즈를 넣어서 만든 인기 있는 한국의 간단한 김밥
2,소고기김밥,김밥,"김에 밥과 소고기, 야채 등을 넣어서 만든 한국의 대표적인 김밥"
3,매콤오징어김밥,김밥,"오징어와 야채를 매콤한 양념으로 볶아 김에 싸서 만든, 한국의 대표적인 매운 김밥"
4,매콤땡+A1초김밥,김밥,"김에 밥과 매콤한 땡초와 함께 참치, 소세지 등을 넣어서 만든, 한국의 대표적인 매..."


In [2]:
from typing import Union, Tuple, List, Dict
from itertools import chain, islice
import torch

def _convert_series_to_list_in_main(series: pd.Series) -> List[List[str]]:

	raw_data = list(series.values)
	return list(chain(*map(lambda x: x.split(), raw_data)))


def extract_keyword_list_in_main(doc: pd.Series, min_count: int = 2, min_length: int = 2) -> List:

	raw_data = _convert_series_to_list_in_main(doc)
	keyword_list = key._extract_keywords(raw_data)
	translated_keyword_list = key._map_english_to_korean(keyword_list)
	refined_keyword_list = key._eliminate_min_count_words(translated_keyword_list, min_count)
	return list(filter(lambda x: len(x) >= min_length, refined_keyword_list))

def create_keyword_embedding_in_main(doc: pd.Series) -> torch.Tensor:

	keyword_list = extract_keyword_list_in_main(doc)
	tokenized_keyword = key.tokenize_keyword(keyword_list)
	return key._create_keyword_embedding(tokenized_keyword)

def create_doc_embedding_in_main(doc: pd.Series) -> torch.Tensor:

	stringified_doc = _convert_series_to_str_in_main(doc)
	tokenized_doc = key.tokenize_keyword(stringified_doc)
	return key._create_doc_embedding(tokenized_doc)

def _convert_series_to_str_in_main(series: pd.Series) -> str:

	return " ".join(list(series.values))

def keyword_(num):
    min_count = 2
    min_length = 1

    doc = scraping_result.iloc[num]
    
    raw_data = _convert_series_to_list_in_main(doc)
    keyword_list = key._extract_keywords(raw_data)
    translated_keyword_list = key._map_english_to_korean(keyword_list)
    refined_keyword_list = key._eliminate_min_count_words(translated_keyword_list, min_count)
    result = list(filter(lambda x: len(x) >= min_length, refined_keyword_list))
    return (result)

In [9]:
min_count = 2
min_length = 1
doc = scraping_result.iloc[0]

#print(f'음식 정보 \n \n {doc} \n \n')

## raw_data = key._convert_series_to_list(doc)
raw_data = _convert_series_to_list_in_main(doc) ## key_extraction 말고 새롭게 선언하면 에러 안 생김 (왜지 도대체) + 단어가 제대로 나뉘지 않아서 수정함

print(f'\n1. 음식 정보를 list로 통합 -> {len(raw_data)} 개 단어')
print(f'{raw_data[:10]}.... \n')
print(raw_data)

keyword_list = key._extract_keywords(raw_data)
print(f'2. 형태소 분석기를 활용해 명사만을 추출 -> {len(keyword_list)} 개 단어')
print(f'{keyword_list[:10]}.... \n')

translated_keyword_list = key._map_english_to_korean(keyword_list)
print(f'3. 영단어를 한글로 변환(ex python -> 파이썬) -> {len(translated_keyword_list)} 개 단어')
print(f'{translated_keyword_list[:10]}.... \n')

refined_keyword_list = key._eliminate_min_count_words(translated_keyword_list, min_count)
print(f'4. 최소 2번이상 반복 사용되는 단어만 추출 -> {len(refined_keyword_list)} 개 단어')
print(f'{refined_keyword_list[:10]}.... \n')

result = list(filter(lambda x: len(x) >= min_length, refined_keyword_list))
print(f'5. 단어 길이가 최소 한개 이상인 단어만 추출 -> {len(result)} 개 단어')
print(f'{result[:10]}.... \n')


1. 음식 정보를 list로 통합 -> 12 개 단어
['야채김밥', '김밥', '김에', '밥과', '다양한', '야채를', '넣어서', '만든', '한국의', '전통적인'].... 

['야채김밥', '김밥', '김에', '밥과', '다양한', '야채를', '넣어서', '만든', '한국의', '전통적인', '간단한', '김밥']
2. 형태소 분석기를 활용해 명사만을 추출 -> 10 개 단어
['야채', '김밥', '김밥', '김', '밥', '다양', '야채', '한국', '전통', '김밥'].... 

3. 영단어를 한글로 변환(ex python -> 파이썬) -> 10 개 단어
['야채', '김밥', '김밥', '김', '밥', '다양', '야채', '한국', '전통', '김밥'].... 

4. 최소 2번이상 반복 사용되는 단어만 추출 -> 2 개 단어
['야채', '김밥'].... 

5. 단어 길이가 최소 한개 이상인 단어만 추출 -> 2 개 단어
['야채', '김밥'].... 



In [10]:
from pprint import pprint

doc = scraping_result.iloc[0]

print(f'-- 메뉴명 -- \n {doc.food_name} \n \n')

keyword_list = extract_keyword_list_in_main(doc)
print(f'음식에 대한 키워드 후보 : {len(result)} 개 단어')
print(f'{result[:10]}.... \n \n')

## keyword_embedding = key.create_keyword_embedding(doc)
keyword_embedding = create_keyword_embedding_in_main(doc)
## doc_embedding = key.create_doc_embedding(doc)
doc_embedding = create_doc_embedding_in_main(doc)

-- 메뉴명 -- 
 야채김밥 
 

음식에 대한 키워드 후보 : 2 개 단어
['야채', '김밥'].... 
 



In [11]:
co_sim_score =key._calc_cosine_similarity(doc_embedding, keyword_embedding).flatten()

keyword = dict(zip(keyword_list, co_sim_score))

sorted_keyword = sorted(keyword.items(), key=lambda k: k[1], reverse=True)

print(f'-- 키워드 추출 결과(20개 요약)--')
pprint(sorted_keyword[:20])

-- 키워드 추출 결과(20개 요약)--
[('야채', 0.85827744), ('김밥', 0.8466986)]


In [12]:

from gensim.models import keyedvectors
import pickle

food_name = []
food_keyword = []

for i in range(len(scraping_result)):
    food_name.append(scraping_result.iloc[i].food_name)
    food_keyword.append(keyword_(i))

print(food_keyword)

# numpy 배열로 변경
#food_name = np.array(food_name)
#food_keyword = np.array(food_keyword, dtype=list)

#food_list = [food_name, food_keyword]

# 굳이 피클로 저장 안 해도 될 듯 (속도 때문에 그런 것 같음)
"""### 피클 파일로 저장 ###
with open("food_data.pickle","wb") as fw:
    pickle.dump(food_list, fw)
 
### 피클 파일 불러오기 ###
with open("food_data.pickle","rb") as fr:
    food_data = pickle.load(fr)

print(food_data)"""

#pd.DataFrame([food_name,food_keyword]).T.head(5)

[['야채', '김밥'], ['치즈', '김밥'], ['소고기', '김밥'], ['오징어', '김밥'], ['김밥'], ['김밥'], ['참치', '와사비', '김밥'], ['새우', '날치', '알', '김밥'], ['어린이', '김밥'], ['찌개'], ['찌개', '돼지고기', '맛'], ['찌개'], ['찌개', '참치', '김치', '맛'], ['고등어', '찌개', '김치', '맛'], ['꽁치', '찌개', '김치', '맛'], ['국'], ['뚝배기', '불고기'], ['김치', '볶음밥', '밥'], ['카레', '덮밥', '밥'], ['덮밥', '밥'], ['참치', '볶음밥'], ['야채', '비빔밥', '밥'], ['밥'], ['햄', '야채', '볶음밥', '밥'], ['새우', '볶음밥', '밥'], ['새우', '날치', '알', '볶음밥', '밥'], ['밥'], ['면'], ['콩나물', '라면'], ['떡', '라면'], ['치즈', '면'], ['만두', '면', '라면'], ['국물'], ['국물'], ['부산', '오뎅'], ['부산', '면'], [], ['김치', '만두', '분식'], ['물'], ['밥'], ['국'], ['떡', '국'], ['면'], ['등심', '돈까스'], ['치즈', '돈까스', '맛'], ['돈까스']]


'### 피클 파일로 저장 ###\nwith open("food_data.pickle","wb") as fw:\n    pickle.dump(food_list, fw)\n \n### 피클 파일 불러오기 ###\nwith open("food_data.pickle","rb") as fr:\n    food_data = pickle.load(fr)\n\nprint(food_data)'

In [7]:
import numpy as np  

# 키워드 검색
search = ['돈까스'] # 입력 -> 키워드
#search = np.array(search)
print('사용자 검색 키워드 : ', search)
print('')

"""# 키워드 확장 
recommand_keyword = w2v_model.most_similar(positive=search, topn=15)
np_recommand_keyword = np.array(list(map(lambda x: x[0], recommand_keyword)))
print('W2V을 활용한 키워드 확장 :', np_recommand_keyword)
print('')"""


# 키워드와 유사한 도서 검색 

user_point = [int(0)] * len(food_name)

for search_key in search:
    for i in range(len(food_name)):
        if search_key in food_keyword[i]:
            user_point[i] = user_point[i] + int(1)


#user_point = np.isin(food_keyword, np.array(search)).sum(axis=1)
#recommand_point = np.isin(food_keyword, np_recommand_keyword).sum(axis=1)

#total_point = (user_point * 3) + recommand_point
total_point = user_point
top_k_idx = np.argsort(total_point)[::-1][:20]

print(type(top_k_idx[0]))

# 메뉴명 연관 점수 저장
food_name = np.array(food_name)
total_point = np.array(total_point)

result  = dict(zip(food_name[top_k_idx], total_point[top_k_idx]))

# 음식 정보 추출
food_info = pd.read_csv('data/food_data.csv',encoding='cp949')
IDX = food_info.food_name.isin(list(result.keys()))

food_recommandation_result = food_info[["food_name", "food_category"]][IDX].sort_values(
    by="food_name", key=lambda x: x.map(result), ascending=False
).reset_index(drop=True)

print('키워드에 따른 상위 20개 음식 추천 결과')
pd.DataFrame(food_recommandation_result)

사용자 검색 키워드 :  ['돈까스']

<class 'numpy.int64'>
키워드에 따른 상위 20개 음식 추천 결과


,food_name,food_category
0,수제왕돈까스,튀김
1,치즈돈까스,튀김
2,등심돈까스,튀김
3,소고기김밥,김밥
4,야채비빔밥,밥
5,참치볶음밥,밥
6,제육덮밥,밥
7,카레덮밥,밥
8,김치볶음밥,밥
9,뚝배기불고기,국
